## Read the script

In [2]:
import os
import numpy as np
import mne
from ieeg.io import get_data, raw_from_layout, update
from ieeg.navigate import trial_ieeg, outliers_to_nan, channel_outlier_marker, crop_empty_data
from ieeg.calc.scaling import rescale
from ieeg.viz.ensemble import chan_grid
from ieeg.timefreq.utils import crop_pad, wavelet_scaleogram
from ieeg.viz.parula import parula_map
save_dir='C:\\Users\\bl314\\Box\\CoganLab\\IndividualMeetings\\Baishen\\ieeg_results\\lexical_delay'

In [3]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
subject = 'D0096'

if subject=='D0053':
    subject_Tag = 'D53'
elif subject=='D0054':
    subject_Tag = 'D54'
elif subject=='D0055':
    subject_Tag = 'D55'
elif subject=='D0057':
    subject_Tag = 'D57'
elif subject=='D0059':
    subject_Tag = 'D59'
elif subject=='D0063':
    subject_Tag = 'D63'
elif subject=='D0069':
    subject_Tag = 'D69'
elif subject=='D0070':
    subject_Tag = 'D70'
elif subject=='D0081':
    subject_Tag = 'D81'
elif subject=='D0094':
    subject_Tag = 'D94'
elif subject=='D0096':
    subject_Tag = 'D96'
elif subject=='D0101':
    subject_Tag = 'D101'
elif subject=='D0102':
    subject_Tag = 'D102'
elif subject=='D0103':
    subject_Tag = 'D103'
elif subject=='D0107':
    subject_Tag = 'D107B'
else:
    print("Subject not found, please check.")

## Plot subj

In [5]:
from ieeg.viz.mri import plot_subj

# plot the subject brain
fig1 = plot_subj(subject_Tag)
#Label every electrode
mne.viz.set_3d_view(fig1, azimuth=150, elevation=70, focalpoint="auto",
                    distance="auto")

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\viz\mri.py:696: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


Could not estimate rigid Talairach alignment, using identity matrix


## Load subj
### <span style="color:red">Please make sure that the **event.tsv** files in the **clean** derivatives do not contain **"BAD boundary"** lines, or remove them if they have</span>

In [4]:
raw = raw_from_layout(layout.derivatives['derivatives/a'], subject=subject, desc='a',extension='.edf',preload=True)

Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-01_desc-a_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-01_desc-a_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-01_desc-a_channels.tsv.


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-02_desc-a_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-02_desc-a_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-02_desc-a_channels.tsv.


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 426 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-02_run-03_desc-a_ieeg.edf...
EDF file detected


...

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 426 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_acq-02_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-02_run-04_desc-a_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-02_run-04_desc-a_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-02_run-04_desc-a_channels.tsv.


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 423 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\ieeg\sub-D0096_acq-02_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Reading 0 ... 11939839  =      0.000 ...  5830.000 secs...


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


### Remove muscle artifacts
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms.html
### <span style="color:red">Please do the muscle artifacts removal according to the wavelet plots and previous cases in sentence repetition</span>

In [5]:
if subject=='D0096':
    eeg_channels_to_exclude=['LTP1', 'LTP2', 'LTP3', 'LTP4', 'LTP5', 'LTP6', 'LTP7', 'LTP8', 'LTP9', 'LTP10','LTAS6', 'LTAS7', 'LTAS8',
 'LTAM1', 'LTAM2', 'LTAM3', 'LTAM4', 'LTAM5', 'LTAM6', 'LTAM7', 'LTAM8', 'LTAM9', 'LTAM10', 'LTAM11', 'LTAM12', 'LTAM13', 'LTAM14',
 'LTMM1', 'LTMM2', 'LTMM3', 'LTMM4', 'LTMM5', 'LTMM6', 'LTMM7', 'LTMM8', 'LTMM9', 'LTMM10', 'LTMM11', 'LTMM12', 'LTMM13', 'LTMM14', 'LTMM15',
 'LFO1', 'LFO2', 'LFO3', 'LFO4', 'LFO5', 'LFO6', 'LFO7', 'LFO8', 'LFO9', 'LFO10', 'LFO11', 'LFO12', 'LFO13', 'LFO14', 'LFO15','LFMM16', 'LFMM17', 'LFMM18','LFAI10']
elif subject=='D0103':
    eeg_channels_to_exclude=['LTP1', 'LTP2', 'LTP3', 'LTP4', 'LTP5', 'LTP6', 'LTP7', 'LTP8', 'LTP9', 'LTP10', 'LTP11', 'LTP12','LTAM1', 'LTAM2', 'LTAM3', 'LTAM4', 'LTAM5', 'LTAM6', 'LTAM7', 'LTAM8', 'LTAM9', 'LTAM10', 'LTAM11', 'LTAM12', 'LTAM13', 'LTAM14', 'LTAM15','LTAS7', 'LTAS8',
 'LTMM2', 'LTMM3', 'LTMM4', 'LTMM5', 'LTMM6', 'LTMM7', 'LTMM8', 'LTMM9', 'LTMM10', 'LTMM11', 'LTMM12', 'LTMM13', 'LTMM14', 'LTMM15','LTMI1', 'LTMI2', 'LTMI3', 'LTMI4', 'LTMI5', 'LTMI6', 'LTMI7', 'LTMI8', 'LTMI9', 'LTMI10', 'LTMI11', 'LTMI12', 'LTMI13', 'LTMI14','LTLI8', 'LTLI9', 'LTLI10', 'LTLI11', 'LTLI12', 'LTLI13', 'LTLI14','LFAM1', 'LFAM2', 'LFAM3', 'LFAM4', 'LFAM5', 'LFAM6', 'LFAM7', 'LFAM8', 'LFAM9', 'LFAM10', 'LFAM11', 'LFAM12', 'LFAM13', 'LFAM14', 'LFAM15','LFAI7', 'LFAI8', 'LFAI9']


raw.info.update(bads=[b for b in eeg_channels_to_exclude if b in raw.ch_names])
update(raw, layout, "muscle")

Processing channel LTP1:
    status: bad
    description: muscle
Processing channel LTP2:
    status: bad
    description: muscle
Processing channel LTP3:
    status: bad
    description: muscle
Processing channel LTP4:
    status: bad
    description: muscle
Processing channel LTP5:
    status: bad
    description: muscle
Processing channel LTP6:
    status: bad
    description: muscle
Processing channel LTP7:
    status: bad
    description: muscle
Processing channel LTP8:
    status: bad
    description: muscle
Processing channel LTP9:
    status: bad
    description: muscle
Processing channel LTP10:
    status: bad
    description: muscle
Processing channel LTAS6:
    status: bad
    description: muscle
Processing channel LTAS7:
    status: bad
    description: muscle
Processing channel LTAS8:
    status: bad
    description: muscle
Processing channel LTAM1:
    status: bad
    description: muscle
Processing channel LTAM2:
    status: bad
    description: muscle
Processing channel 

## Re-referenced to the average

In [6]:
ch_type = raw.get_channel_types(only_data_chs=True)[0]
raw.set_eeg_reference(ref_channels="average", ch_type=ch_type)

Applying average reference.
Applying a custom ('sEEG',) reference.


<RawEDF | sub-D0096_task-LexicalDecRepDelay_acq-01_run-01_desc-a_ieeg.edf, 232 x 11939840 (5830.0 s), ~20.64 GB, data loaded>

## Multitaper spectrogram

In [7]:
if not os.path.exists(os.path.join(save_dir, subject)):
    os.mkdir(os.path.join(save_dir, subject)) 
if not os.path.exists(os.path.join(save_dir, subject,'multitaper')):
    os.mkdir(os.path.join(save_dir, subject,'multitaper'))

In [8]:
# Wavelet is good to detect and remove muscle artifact channels
# Also plot the subject brain
for task, task_Tag in zip(('Repeat','Yes_No'),('Rep','YN')):
    for epoch, t, tag in zip(
        ('Auditory_stim/'+task+'/CORRECT','Delay/'+task+'/CORRECT','Resp/'+task+'/CORRECT'),
        ((-0.5, 1.5),(-0.5, 1.5),(-0.5, 1)),
        ('Auditory-'+task_Tag,'Delay-'+task_Tag,'Resp-'+task_Tag)
    ):
        
        # Get the spectras
        t1 = t[0] - 0.5
        t2 = t[1] + 0.5
        times = (t1, t2)
        trials = trial_ieeg(raw, epoch, times, preload=True)
        outliers_to_nan(trials, outliers=10)
    
        
        ##############################
        ####### Multitapper ##########
        ##############################
        
        freq = np.linspace(0.5, 200, num=80)
        kwargs = dict(average=False, n_jobs=-3, freqs=freq, return_itc=False,
                      n_cycles=freq/2, time_bandwidth=4,
                      # n_fft=int(trials.info['sfreq'] * 2.75),
                      decim=20, )
                      # adaptive=True)
        spectra_multitaper = trials.compute_tfr(method="multitaper",  **kwargs)
        crop_pad(spectra_multitaper, "0.5s") # cut the first and final 0.5s, change to zero
        
                # Get the baseline
        if epoch=='Auditory_stim/'+task+'/CORRECT':
            base_multitaper = spectra_multitaper.copy().crop(-0.5, 0)
            base_multitaper = base_multitaper.average(lambda x: np.nanmean(x, axis=0), copy=True)
 
        # Baseline correction
        spectra_multitaper = spectra_multitaper.average(lambda x: np.nanmean(x, axis=0), copy=True)   
        spectra_multitaper = rescale(spectra_multitaper, base_multitaper, copy=True, mode='ratio')
        
        # Save spectras
        filename = os.path.join(save_dir, subject,'multitaper',f'{tag}-tfr.h5')
        mne.time_frequency.write_tfrs(filename, spectra_multitaper, overwrite=True)
        
        # Make spectrograms
        chan_grids=chan_grid(spectra_multitaper, size = (20,10),vlim=(0.7, 1.4), cmap=parula_map)
        
        # Save spectrograms
        fig_count=0
        for fig in chan_grids:
            figdir = os.path.join(save_dir, subject,'multitaper',f'{tag}_{fig_count+1}.jpg')
            chan_grids[fig_count].savefig(figdir,dpi=300)   
            fig_count+=1
        
        del trials,spectra_multitaper,filename
    del base_multitaper

Used Annotations descriptions: [np.str_('Auditory_stim/Repeat/Nonword/banic/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/berin/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/boril/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/caris/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/cazel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/delin/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/devim/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/galel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/gapel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/gorel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/gorel/EARLY_RESP'), np.str_('Auditory_stim/Repeat/Nonword/havel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/havel/ERR_TASK/REP_YN'), np.str_('Auditory_stim/Repeat/Nonword/herib/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/herib/EARLY_RESP'), np.str_('Auditory_stim/Repeat/Nonword/janem/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/japel/CORRECT'), np.str_('Auditory_s

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done  42 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-3)]: Done 132 tasks      | elapsed:   34.7s
[Parallel(n_jobs=-3)]: Done 171 out of 171 | elapsed:   40.3s finished
C:\Users\bl314\AppData\Local\Temp\ipykernel_34560\3313998584.py:33: RuntimeWarning: tmin is not in time interval. tmin is set to <class 'mne.time_frequency.tfr.EpochsTFR'>.tmin (-0.492188 s)
  base_multitaper = spectra_multitaper.copy().crop(-0.5, 0)


Applying baseline correction (mode: ratio)
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction appl

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done  42 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-3)]: Done 132 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-3)]: Done 171 out of 171 | elapsed:   36.6s finished


Applying baseline correction (mode: ratio)
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction appl

...

Applying baseline correction (mode: ratio)
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction appl

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done  42 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-3)]: Done 132 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-3)]: Done 171 out of 171 | elapsed:   33.2s finished


Applying baseline correction (mode: ratio)
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction appl

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-3)]: Done  42 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-3)]: Done 132 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-3)]: Done 171 out of 171 | elapsed:   28.0s finished


Applying baseline correction (mode: ratio)
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction appl